In [60]:
# Import Pyspark
import pyspark
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, when, count, col, countDistinct

In [6]:
# Establish Spark Session
spark = SparkSession \
    .builder \
    .appName("Dataset Investigation") \
    .getOrCreate()

In [38]:
# Load First Dataset
df_1_path = "data/df_test"
df_1 = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv(df_1_path)

# Load Second Dataset
df_2_path = "data//df_result"
df_2 = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv(df_2_path)

In [13]:
# Check Schema for df_1

In [39]:
df_1.printSchema()

root
 |-- test_id: integer (nullable = true)
 |-- rfr_id: integer (nullable = true)
 |-- rfr_type_code: string (nullable = true)
 |-- location_id: integer (nullable = true)
 |-- dangerous_mark: string (nullable = true)



In [15]:
# Check Schema for df_2

In [40]:
df_2.printSchema()

root
 |-- test_id: integer (nullable = true)
 |-- vehicle_id: integer (nullable = true)
 |-- test_date: string (nullable = true)
 |-- test_class_id: integer (nullable = true)
 |-- test_type: string (nullable = true)
 |-- test_result: string (nullable = true)
 |-- test_mileage: integer (nullable = true)
 |-- postcode_area: string (nullable = true)
 |-- make: string (nullable = true)
 |-- model: string (nullable = true)
 |-- colour: string (nullable = true)
 |-- fuel_type: string (nullable = true)
 |-- cylinder_capacity: integer (nullable = true)
 |-- first_use_date: string (nullable = true)



In [22]:
# Review top 10 rows of df_1

In [41]:
df_1.show(n=10)

+---------+------+-------------+-----------+--------------+
|  test_id|rfr_id|rfr_type_code|location_id|dangerous_mark|
+---------+------+-------------+-----------+--------------+
|666422869| 31380|            A|         25|          null|
|666422869| 31194|            A|         25|          null|
|623774383| 31194|            A|          9|          null|
|325234981| 40368|            F|          9|          null|
|325234981| 40376|            F|          9|          null|
|325234981| 40580|            F|          9|             D|
|325234981| 40206|            A|          9|          null|
|367883467| 31194|            A|          7|          null|
|367883467| 31322|            A|          7|          null|
|154641037| 30856|            M|          1|          null|
+---------+------+-------------+-----------+--------------+
only showing top 10 rows



In [24]:
# Review top 10 rows of df_2

In [42]:
df_2.show(n=10)

+---------+----------+----------+-------------+---------+-----------+------------+-------------+--------+--------+------+---------+-----------------+--------------+
|  test_id|vehicle_id| test_date|test_class_id|test_type|test_result|test_mileage|postcode_area|    make|   model|colour|fuel_type|cylinder_capacity|first_use_date|
+---------+----------+----------+-------------+---------+-----------+------------+-------------+--------+--------+------+---------+-----------------+--------------+
|666422869|1253657552|2020-01-01|            4|       NT|          P|       63975|           TR| CITROEN|DISPATCH| WHITE|       DI|             1560|    2011-03-14|
|623774383|  51021182|2020-01-01|            4|       NT|          P|      107361|           NN|    SEAT|   IBIZA|YELLOW|       PE|             1390|    2008-12-18|
|581125897| 612989654|2020-01-01|            4|       NT|          P|       73160|           NN|MERCEDES|   A 150|SILVER|       PE|             1498|    2007-09-28|
|538477411

In [ ]:
# Review summary statistics for df_1

In [43]:
df_1.describe().show()

+-------+-------------------+-----------------+-------------+------------------+--------------+
|summary|            test_id|           rfr_id|rfr_type_code|       location_id|dangerous_mark|
+-------+-------------------+-----------------+-------------+------------------+--------------+
|  count|           75907074|         75907074|     75907074|          75907074|       3946877|
|   mean|9.999528644262294E8|30332.30882541988|         null|20.441835737206784|          null|
| stddev|5.773192977866449E8|4292.207100348532|         null| 28.95789023526161|          null|
|    min|                 13|            10001|            A|                 1|             D|
|    max|         1999999879|            40668|            P|               257|             D|
+-------+-------------------+-----------------+-------------+------------------+--------------+



In [ ]:
# Review summary statistics for df_2

In [44]:
df_2.describe().show()

+-------+-------------------+-------------------+----------+------------------+---------+-----------+-----------------+-------------+---------+--------+--------+---------+-----------------+--------------+
|summary|            test_id|         vehicle_id| test_date|     test_class_id|test_type|test_result|     test_mileage|postcode_area|     make|   model|  colour|fuel_type|cylinder_capacity|first_use_date|
+-------+-------------------+-------------------+----------+------------------+---------+-----------+-----------------+-------------+---------+--------+--------+---------+-----------------+--------------+
|  count|           38594013|           38594013|  38594013|          38594013| 38594013|   38594013|         37586721|     38594013| 38594013|38594013|38594013| 38594013|         38516999|      38593382|
|   mean| 9.99981898874274E8|7.499174761990405E8|      null|4.0257636333386735|     null|       null|74862.38603199784|         null| Infinity|Infinity|    null|     null|1714.3848

In [51]:
# Investigate Null/NaN values in df_1

In [56]:
df_1.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_1.columns]).show()

+-------+------+-------------+-----------+--------------+
|test_id|rfr_id|rfr_type_code|location_id|dangerous_mark|
+-------+------+-------------+-----------+--------------+
|      0|     0|            0|          0|      71960197|
+-------+------+-------------+-----------+--------------+



In [54]:
# Investigate Null/NaN values in df_2

In [52]:
df_2.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_2.columns]).show()

+-------+----------+---------+-------------+---------+-----------+------------+-------------+----+-----+------+---------+-----------------+--------------+
|test_id|vehicle_id|test_date|test_class_id|test_type|test_result|test_mileage|postcode_area|make|model|colour|fuel_type|cylinder_capacity|first_use_date|
+-------+----------+---------+-------------+---------+-----------+------------+-------------+----+-----+------+---------+-----------------+--------------+
|      0|         0|        0|            0|        0|          0|     1007292|            0|   0|    0|     0|        0|            77014|           631|
+-------+----------+---------+-------------+---------+-----------+------------+-------------+----+-----+------+---------+-----------------+--------------+



In [64]:
# Check that test_id is a primary key in df_2

In [63]:
df_2.select(countDistinct("test_id")).show()

+-----------------------+
|count(DISTINCT test_id)|
+-----------------------+
|               38594013|
+-----------------------+

